In [2]:
from glob import glob
from os.path import join
import pandas as pd
import numpy as np

In [3]:
common_path = '/Volumes/MMIS-Saraiv/Datasets/Miltiadous Dataset/features'
all_directories = glob(join(common_path, '*'))
all_directories = [directory for directory in all_directories if not directory.endswith('.csv')]

In [4]:
FEATURES_SELECTED = ['Spectral#RelativePower#C3#beta1', 'Spectral#EdgeFrequency#C3#beta3', 'Spectral#RelativePower#C3#gamma', 'Spectral#EdgeFrequency#C4#alpha1', 'Spectral#RelativePower#C4#beta3', 'Spectral#EdgeFrequency#C4#beta3', 'Spectral#EdgeFrequency#C4#gamma', 'Spectral#Flatness#Cz#theta', 'Spectral#PeakFrequency#Cz#theta', 'Spectral#EdgeFrequency#Cz#beta3', 'Spectral#EdgeFrequency#Cz#gamma', 'Spectral#PeakFrequency#Cz#gamma', 'Spectral#RelativePower#F3#beta1', 'Spectral#Diff#F4#delta', 'Spectral#RelativePower#F7#beta3', 'Spectral#EdgeFrequency#F7#beta3', 'Spectral#RelativePower#F7#gamma', 'Spectral#RelativePower#F8#beta1', 'Spectral#EdgeFrequency#F8#beta3', 'Spectral#RelativePower#Fp1#beta1', 'Spectral#EdgeFrequency#Fp1#beta3', 'Spectral#Diff#Fp2#delta', 'Spectral#RelativePower#Fp2#beta1', 'Spectral#RelativePower#Fp2#beta3', 'Spectral#Diff#Fpz#beta2', 'Spectral#Entropy#O1#delta', 'Spectral#RelativePower#O1#beta2', 'Spectral#EdgeFrequency#O1#beta2', 'Spectral#EdgeFrequency#O1#beta3', 'Spectral#RelativePower#O2#delta', 'Spectral#PeakFrequency#O2#alpha1', 'Spectral#RelativePower#O2#beta1', 'Spectral#RelativePower#O2#beta3', 'Spectral#Diff#P3#beta1', 'Spectral#RelativePower#P3#beta3', 'Spectral#RelativePower#Pz#alpha1', 'Spectral#EdgeFrequency#Pz#beta3', 'Spectral#RelativePower#T4#alpha1', 'Spectral#RelativePower#T4#beta3', 'Spectral#RelativePower#T4#gamma', 'Spectral#EdgeFrequency#T5#beta2', 'Hjorth#Complexity#T5', 'Hjorth#Complexity#P4', 'Hjorth#Complexity#F7', 'Hjorth#Complexity#T4', 'Hjorth#Complexity#F8', 'Hjorth#Complexity#T3', 'Hjorth#Mobility#P3', 'PLI#Frontal(L)-Temporal(R)#alpha1', 'PLI#Frontal(L)-Occipital(L)#alpha1', 'PLI#Frontal(R)-Temporal(R)#alpha1', 'PLI#Temporal(R)-Parietal(R)#alpha1', 'PLI#Temporal(R)-Occipital(L)#alpha1', 'PLI#Parietal(R)-Occipital(L)#alpha1', 'PLI#Occipital(L)-Occipital(R)#alpha1', 'PLI#Temporal(R)-Occipital(R)#alpha2', 'PLI#Parietal(R)-Occipital(L)#alpha2', 'COH#Frontal(L)-Frontal(R)#theta', 'COH#Frontal(L)-Occipital(L)#theta', 'COH#Frontal(L)-Occipital(R)#alpha1', 'COH#Frontal(R)-Occipital(L)#alpha1', 'COH#Parietal(R)-Occipital(L)#alpha1', 'COH#Frontal(L)-Frontal(R)#alpha2', 'COH#Frontal(L)-Occipital(R)#alpha2', 'COH#Parietal(R)-Occipital(L)#alpha2', 'COH#Parietal(R)-Occipital(R)#alpha2', 'COH#Occipital(L)-Occipital(R)#alpha2', 'COH#Frontal(L)-Occipital(L)#beta1', 'COH#Temporal(R)-Parietal(R)#beta1', 'COH#Parietal(R)-Occipital(R)#beta1', 'COH#Frontal(L)-Parietal(L)#beta2', 'COH#Frontal(R)-Occipital(L)#beta2', 'COH#Frontal(L)-Temporal(R)#beta3', 'COH#Frontal(L)-Parietal(L)#beta3', 'COH#Frontal(L)-Occipital(L)#beta3', 'COH#Frontal(L)-Occipital(R)#beta3', 'COH#Frontal(R)-Occipital(L)#beta3', 'COH#Temporal(L)-Occipital(R)#beta3', 'COH#Frontal(L)-Occipital(R)#gamma', 'COH#Frontal(R)-Occipital(R)#gamma']


In [5]:
underrepresented_targets = [4, 6, 9]

In [6]:
# Read all targets
all_targets = pd.read_csv('/Volumes/MMIS-Saraiv/Datasets/Miltiadous Dataset/participants.tsv', index_col=0, sep='\t')
# Remove "sub-" from the index and make them integers
all_targets.index = all_targets.index.str.replace('sub-', '')
all_targets.index = all_targets.index.astype(int)
all_targets

In [18]:
# Read all and standardize
all_features = []
for subject_path in all_directories:
    subject = subject_path.split('/')[-1]
        
    # Get feature files
    all_files = glob(join(subject_path, '*.csv'))
    # remove all files that have "Multiple" in their name
    all_files = [file for file in all_files if 'Multiple' not in file]
    # Read them all
    all_data = [pd.read_csv(file, index_col=0) for file in all_files]
    # FIX: the spectral data has a different index
    for data in all_data:
        if 'Spectral' in data.columns[0]:
            data.index = [int(subject),]
    # Concatenate all their columns in a single DataFrame
    all_data = pd.concat(all_data, axis=1)
    # Keep only selected features
    selected_features = all_data[FEATURES_SELECTED]
    # drop nans
    selected_features = selected_features.dropna()
    
    all_features.append(selected_features)
    
all_features = pd.concat(all_features, axis=0)

# Standardize feature-wise
#all_features = (all_features - all_features.mean()) / all_features.std()
all_features

In [24]:
all_features_not_norm = all_features.copy()

In [25]:
# Standardize feature-wise
all_features = (all_features_not_norm - all_features_not_norm.mean()) / all_features_not_norm.std()

In [30]:
# Normalize feature-wise
all_features = (all_features_not_norm - all_features_not_norm.min()) / (all_features_not_norm.max() - all_features_not_norm.min())

In [31]:
# Keep only sessions that have MMSE in 'underrepresented_targets'
single_examples = all_features.loc[all_targets[all_targets['MMSE'].isin(underrepresented_targets)].index]
single_examples

In [33]:
from scipy.stats import kruskal, mannwhitneyu, wilcoxon, ttest_ind
measure = wilcoxon
# Compute statistical test between each single example and the rest of the examples
for i, (index, example) in enumerate(single_examples.iterrows()):
    print(f"Example of target {all_targets.loc[index]['MMSE']}")
    avg_stat_same, avg_stat_diff = [], []
    avg_p_same, avg_p_diff = [], []
    n_same, n_diff = 0, 0
    for j, (index2, example2) in enumerate(all_features.iterrows()):
        if index == index2:
            continue
        #u_stat, p = mannwhitneyu(example, example2)
        # Kruskal-Wallis H-test
        stat, p = measure(example, example2)
        statement = f"With example of target {all_targets.loc[index2]['MMSE']}: "
        if p > 0.05:
            avg_stat_same.append(stat)
            avg_p_same.append(p)
            n_same += 1
        else:
            n_diff += 1
            avg_stat_diff.append(stat)
            avg_p_diff.append(p)
        #print(statement + f"they come from the same subject (p={p:.3f})." if p > 0.05 else statement + f"they come from different subjects (p={p:.3f}).")
    print(f"Same: {n_same}, Different: {n_diff}")
    print(f"Average statistic same: {np.mean(avg_stat_same):.3f}, Average p same: {np.mean(avg_p_same):.3f}")
    print(f"Average statistic diff: {np.mean(avg_stat_diff):.3f}, Average p diff: {np.mean(avg_p_diff):.3f}")
    print("-------")

In [37]:
measure = mannwhitneyu
# Compute statistical test between example with itself and others, considering only 26 <= MMSE <= 30
healthy_examples = all_features.loc[all_targets[all_targets['MMSE'] >= 30].index]
for i, (index, example) in enumerate(healthy_examples.iterrows()):
    print(f"Example of target {all_targets.loc[index]['MMSE']}")
    avg_stat_same, avg_stat_diff = [], []
    avg_p_same, avg_p_diff = [], []
    n_same, n_diff = 0, 0
    for j, (index2, example2) in enumerate(all_features.iterrows()):
        if index == index2:
            print("With itself:")
            stat, p = measure(example, example2)
            print(f"Statistic: {stat:.3f}, p: {p:.3f}")
        else:
            stat, p = measure(example, example2)
            statement = f"With example of target {all_targets.loc[index2]['MMSE']}: "
            if p > 0.05:
                avg_stat_same.append(stat) 
                avg_p_same.append(p)
                n_same += 1
            else:
                n_diff += 1
                avg_stat_diff.append(stat)
                avg_p_diff.append(p)
    print(f"Same: {n_same}, Different: {n_diff}")
    print(f"Average statistic same: {np.mean(avg_stat_same):.3f}, Average p same: {np.mean(avg_p_same):.3f}")
    print(f"Average statistic diff: {np.mean(avg_stat_diff):.3f}, Average p diff: {np.mean(avg_p_diff):.3f}")
    print("-------")

In [36]:
from scipy.stats import mannwhitneyu


# Data augmentation by iteratively interpolating examples with targets in between 4 and 6 (exclusive) and 6 and 9 (exclusive)
# (4, 6) -> 5; then (4, 6) -> 4.5 and (5, 6) -> 5.5; and so on exponentially, until the distance between the targets is equal or less than 0.1
# (6, 9) -> 7.5; then (6, 9) -> 8.25 and (7.5, 9) -> 8.75; and so on exponentially, until the distance between the targets is equal or less than 0.1
def exponentially_generate_synthetic_examples(a, target_a, b, target_b, min_difference):
    # Generate the middle target
    middle_target = (target_a + target_b) / 2
    # Generate the middle example by interpolating the feature vectors
    middle_example = (a + b) / 2
    # Apply gaussian noise to the middle example
    #middle_example += np.random.normal(0, 0.15, middle_example.shape)
    # Add the middle example to the list
    synthetic_examples = [(middle_example, middle_target)]

    # If the difference between the targets is less than the minimum difference, return the synthetic examples
    if abs(target_a - target_b) <= min_difference:
        return synthetic_examples
    
    # Otherwise, recursively generate the synthetic examples
    synthetic_examples += exponentially_generate_synthetic_examples(a, target_a, middle_example, middle_target, min_difference)
    synthetic_examples += exponentially_generate_synthetic_examples(middle_example, middle_target, b, target_b, min_difference)
    return synthetic_examples




In [37]:
a, b = single_examples.loc[single_examples.index[0]], single_examples.loc[single_examples.index[1]]
target_a, target_b = all_targets.loc[single_examples.index[0]]['MMSE'], all_targets.loc[single_examples.index[1]]['MMSE']

synthetic = exponentially_generate_synthetic_examples(a, target_a, b, target_b, min_difference=0.5)

# compute the statistical test between the two original examples
u_stat_ref, p_ref = mannwhitneyu(a, b)
statement = f"With original examples of targets {target_a} and {target_b}: "
print(statement + f"they come from the same subject (p={p_ref:.3f})." if p_ref > 0.05 else statement + f"they come from different subjects (p={p_ref:.3f}).")
print("-------")

# compute the statistical test between every synthetic example and the original examples
res = {}
for example in synthetic:
    vector, target = example
    print("Synthetic example of target", target)
    u_stat_a, p_a = mannwhitneyu(a, example[0])
    u_stat_b, p_b = mannwhitneyu(b, example[0])
    statement = f"With parent example of target {target_a}: "
    print(statement + f"they come from the same subject (p={p_a:.3f})." if p_a > 0.05 else statement + f"they come from different subjects (p={p_a:.3f}).")
    statement = f"With parent example of target {target_b}: "
    print(statement + f"they come from the same subject (p={p_b:.3f})." if p_b > 0.05 else statement + f"they come from different subjects (p={p_b:.3f}).")
    print("-------")
    res[target] = (p_a>0.05, p_b>0.05)

# Draw a plot illustrating the tendency of the synthetic examples to be closer to the target, using res
# | ------------------------------------------------------- |
# |    <-    |    <-   |    <-  ->    |    ->    |    ->    |   
# | ------------------------------------------------------- |
#     4.25      4.5      4.75    5.0      5.25     5.5

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 3))
close_to_left = [target for target, (left, right) in res.items() if left]
close_to_right = [target for target, (left, right) in res.items() if right]
plt.scatter([x - 0.01 for x in close_to_left], [0] * len(close_to_left), label='Assumed to the same subject as the 4',
            color='blue')
plt.scatter([x + 0.01 for x in close_to_right], [0] * len(close_to_right), label='Assumed to the same subject as the 6',
            color='red')
plt.legend()
plt.show()

In [41]:
a, b = single_examples.loc[single_examples.index[0]], single_examples.loc[single_examples.index[2]]
target_a, target_b = all_targets.loc[single_examples.index[0]]['MMSE'], all_targets.loc[single_examples.index[2]]['MMSE']

synthetic = exponentially_generate_synthetic_examples(a, target_a, b, target_b, min_difference=0.5)

# compute the statistical test between the two original examples
u_stat_ref, p_ref = mannwhitneyu(a, b)
statement = f"With original examples of targets {target_a} and {target_b}: "
print(statement + f"they come from the same subject (p={p_ref:.3f})." if p_ref > 0.05 else statement + f"they come from different subjects (p={p_ref:.3f}).")
print("-------")

# compute the statistical test between every synthetic example and the original examples
res = {}
for example in synthetic:
    vector, target = example
    print("Synthetic example of target", target)
    u_stat_a, p_a = mannwhitneyu(a, example[0])
    u_stat_b, p_b = mannwhitneyu(b, example[0])
    statement = f"With parent example of target {target_a}: "
    print(statement + f"they come from the same subject (p={p_a:.3f})." if p_a > 0.05 else statement + f"they come from different subjects (p={p_a:.3f}).")
    statement = f"With parent example of target {target_b}: "
    print(statement + f"they come from the same subject (p={p_b:.3f})." if p_b > 0.05 else statement + f"they come from different subjects (p={p_b:.3f}).")
    print("-------")
    res[target] = (p_a>0.05, p_b>0.05)

# Draw a plot illustrating the tendency of the synthetic examples to be closer to the target, using res
# | ------------------------------------------------------- |
# |    <-    |    <-   |    <-  ->    |    ->    |    ->    |   
# | ------------------------------------------------------- |
#     4.25      4.5      4.75    5.0      5.25     5.5

import matplotlib.pyplot as plt

plt.figure(figsize=(15, 2))
close_to_left = [target for target, (left, right) in res.items() if left]
close_to_right = [target for target, (left, right) in res.items() if right]
plt.scatter([x - 0.01 for x in close_to_left], [0] * len(close_to_left), label='Assumed to the same subject as the 6',
            color='blue')
plt.scatter([x + 0.01 for x in close_to_right], [0] * len(close_to_right), label='Assumed to the same subject as the 9',
            color='red')
plt.legend()
plt.show()